# Visualize Planet Composite Images

In [ ]:
import ee
ee.Initialize()

In [ ]:
# Import Libraries
import geemap
import re
from google.cloud import storage
from pprint import pprint

In [ ]:
# Set up access to abrupt_thaw
storage_client = storage.Client(project="AbruptThawMapping")
abrupt_thaw = storage_client.get_bucket('abrupt_thaw')

## Define Functions

In [ ]:
# apply image IDs to each image in the collection
def setID(image):
    img_id = image.id();
    img_prop = image.setMulti({'ID': img_id});
    img_prop = ee.Image(img_prop);
    return img_prop;

In [ ]:
# Mask the data to values greater than 0 (which is how nodata gets imported into GEE by default)
def mask_0(image):
    mask = image.gt(0)
    return image.updateMask(mask)

## Import Data

In [ ]:
# Import Planet Data
planet = ee.ImageCollection('projects/abruptthawmapping/assets/nitze_regions_composites')
planet = planet.map(setID)

# Import Sentinel Data
s2 = ee.ImageCollection('projects/abruptthawmapping/assets/nitze_regions_sentinel_reference')
s2 = s2.map(setID)

# Polygons
polygons = ee.FeatureCollection('users/gfiske/FrostCraters/rts_polygons_in_Nitze_areas_for_Yili_Mar_2022')

In [ ]:
# Filter to look at composites which include both 2019 and 2018 images
planet = planet.filter(ee.Filter(ee.Filter.stringContains('ID', 'all'))).median()
# pprint(planet.aggregate_array('ID').getInfo())

In [ ]:
# Prep Map
Map = geemap.Map()
Map.centerObject(planet)

In [ ]:
# Set visualization parameters
vis_params = {
    'min': 0,'max': 1200,
    'bands': ['b3', 'b2', 'b1'],
    'gamma': 1
}

In [ ]:
# Add layers
# Add Sentinel-2 composite
Map.addLayer(s2.median(), {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':1200}, 'S2 All Years')

# add Planet composites
Map.addLayer(planet, vis_params, 'Planet')

# add RTS Polygons
Map.addLayer(polygons, {'color': 'red'}, 'RTS')

In [ ]:
# Display the map
Map